In [1]:
from process_bigraph import Composite 

from biosimulators_processes import CORE 

Cannot register SimulariumSmoldynStep. Error:
**
No module named 'simulariumio'
**
Cannot register MongoDatabaseEmitter. Error:
**
No module named 'simulariumio'
**


In [2]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

doc = {
    'dFBA': {
        '_type': 'process',
        'address': 'local:dfba-process',
        'config': {
            'model': {
                'model_source': model_fp
            },
            'simulator': 'copasi',
            'start': 0,
            'stop': 10,
            'steps': 100
        },
        'inputs': {},
        'outputs': {
            'solution': ['solution_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'solution': 'tree[float]'
            }
        },
        'inputs': {
            'solution': ['solution_store']
        }
    }
}


spec = doc.copy()

In [ ]:
comp = Composite(
    config={'state': spec},
    core=CORE
)

In [ ]:
comp.run(3)

In [7]:
import numpy as np
import os 
from pathlib import Path
from tqdm import tqdm

from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import cobra
from cobra.io import load_model as load_cobra, read_sbml_model
from basico import * 

from biosimulators_processes.helpers import generate_reaction_mappings


# set up models
model_file = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

# set up cobra model
data_dir = Path(os.path.dirname(model_file))
path = data_dir / model_file.split('/')[-1]
cobrapy = read_sbml_model(str(path.resolve()))  # load_cobra('textbook')

# set up copasi model
copasi = load_model(model_fp)


Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction4 "translation of LacI">'
Missing upper flux bound se

In [13]:
s = get_species(model=copasi)

s.concentration.to_dict()

{'LacI protein': 0.0,
 'TetR protein': 0.0,
 'cI protein': 0.0,
 'LacI mRNA': 0.0,
 'TetR mRNA': 20.0,
 'cI mRNA': 0.0}

In [ ]:
def _add_dynamic_bounds(fba_model, utc_model, y):
    # 1. get reaction mappings 
    # 2. y = get_species(model=copasi).initial_concentration.values
    # 3. for each species in y, calculate max import (TODO: make this specific)
    # 4. for each species mapping, use the dict val (reaction name) to say fba_model.reactions.get_by_id(reaction_name) = max_import
    biomass, glucose = y 
    glucose_max_import = -10 * glucose / (5 + glucose)
    fba_model.reactions.EX_glc__D_e.lower_bound = glucose_max_import


def add_dynamic_bounds(fba_model, utc_model, y, mappings, species_names):
    """
    Update reaction bounds dynamically based on species concentrations using mappings.

    Parameters:
        fba_model: The FBA model (e.g., COBRA model)
        utc_model: The UTC model (e.g., COPASI model)
        y: Array of species concentrations at the current time point
        mappings: List of mappings from species names to reactions
        species_names: List of species names corresponding to y
    
    # 1. get reaction mappings 
    # 2. y = get_species(model=copasi).initial_concentration.values
    # 3. for each species in y, calculate max import (TODO: make this specific)
    # 4. for each species mapping, use the dict val (reaction name) to say fba_model.reactions.get_by_id(reaction_name) = max_import
    """
    # Zip species names and their corresponding concentrations for iteration
    species_concentrations = dict(zip(species_names, y))

    # Iterate through the mappings to apply species-specific logic
    for mapping in mappings:
        # Each mapping is a dictionary like {'LacI mRNA': 'degradation of LacI transcripts'}
        for species, reaction_name in mapping.items():
            if species in species_concentrations:
                concentration = species_concentrations[species]
                
                # Example: Set bounds or apply specific logic based on reaction type
                if "degradation" in reaction_name:
                    # Apply logic for degradation reactions
                    max_import = -10 * concentration / (5 + concentration)
                    fba_model.reactions.get_by_id(reaction_name).lower_bound = max_import
                elif "transcription" in reaction_name:
                    # Apply logic for transcription reactions
                    max_import = 5 * concentration / (3 + concentration)
                    fba_model.reactions.get_by_id(reaction_name).lower_bound = max_import
                elif "translation" in reaction_name:
                    # Apply logic for translation reactions
                    max_import = 8 * concentration / (4 + concentration)
                    fba_model.reactions.get_by_id(reaction_name).lower_bound = max_import
                
                # Add additional logic for other types of reactions if needed


def dynamic_system(t, y, fba_model, utc_model, mappings, species_names):
    """
    Calculate the time derivative of external species using specific reaction mappings.

    Parameters:
        t: Current time
        y: Array of species concentrations at time t
        fba_model: The FBA model (e.g., COBRA model)
        utc_model: The UTC model (e.g., COPASI model)
        mappings: List of mappings from species names to reactions
        species_names: List of species names corresponding to y

    Returns:
        Fluxes calculated based on current concentrations
    """
    # Update the FBA model's reaction bounds using species concentrations and mappings
    add_dynamic_bounds(fba_model, utc_model, y, mappings, species_names)

    # Run FBA with updated bounds and calculate fluxes
    cobra.util.add_lp_feasibility(fba_model)
    feasibility = cobra.util.fix_objective_as_constraint(fba_model)
    
    # Example of reactions to optimize (can vary based on your specific model)
    reaction_list = ['EX_glc__D_e', 'Biomass_Ecoli_core']  # Example reactions
    lex_constraints = cobra.util.add_lexicographic_constraints(
        fba_model, reaction_list, ['max', 'max']
    )

    # Handle the calculated fluxes
    # Check if biomass is one of the species and handle accordingly
    if 'biomass' in species_names:
        # If biomass is one of the species, use it to scale the fluxes
        biomass_index = species_names.index('biomass')
        biomass_concentration = y[biomass_index]
        fluxes = lex_constraints.values * biomass_concentration
    else:
        # If biomass is not part of the species, return the raw fluxes
        fluxes = lex_constraints.values

    # Update progress bar (optional)
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description(f't = {t:.3f}')

    return fluxes


def _dynamic_system(t, y):
    """Calculate the time derivative of external species."""
    # 1. run copasi utc (update model!)
    # 2. run get_species(model=copasi).concentration.to_dict().values()
    # 3. assign these to y = #2
    # 4. iteratively run: for species in y: DO THE REST
    biomass, glucose = y
    
    with cobrapy:
        add_dynamic_bounds(cobrapy, copasi, y)

        cobra.util.add_lp_feasibility(cobrapy)
        feasibility = cobra.util.fix_objective_as_constraint(cobrapy)
        lex_constraints = cobra.util.add_lexicographic_constraints(
            cobrapy, ['Biomass_Ecoli_core', 'EX_glc__D_e'], ['max', 'max'])

    fluxes = lex_constraints.values
    fluxes *= biomass

    # This implementation is **not** efficient, so I display the current
    # simulation time using a progress bar.
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description('t = {:.3f}'.format(t))

    return fluxes

dynamic_system.pbar = None


def infeasible_event(t, y):
    """
    Determine solution feasibility.

    Avoiding infeasible solutions is handled by solve_ivp's built-in event detection.
    This function re-solves the LP to determine whether or not the solution is feasible
    (and if not, how far it is from feasibility). When the sign of this function changes
    from -epsilon to positive, we know the solution is no longer feasible.

    """

    with cobrapy:

        add_dynamic_bounds(cobrapy, y)

        cobra.util.add_lp_feasibility(cobrapy)
        feasibility = cobra.util.fix_objective_as_constraint(cobrapy)

    return feasibility - infeasible_event.epsilon

infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

In [15]:
generate_reaction_mappings(get_species(model=copasi).index.tolist(), cobrapy)

[{'LacI mRNA': 'degradation of LacI transcripts'},
 {'TetR mRNA': 'degradation of TetR transcripts'},
 {'cI mRNA': 'degradation of CI transcripts'},
 {'LacI protein': 'translation of LacI'},
 {'TetR protein': 'translation of TetR'},
 {'cI protein': 'translation of CI'},
 {'LacI protein': 'degradation of LacI'},
 {'TetR protein': 'degradation of TetR'},
 {'cI protein': 'degradation of CI'},
 {'LacI mRNA': 'transcription of LacI'},
 {'TetR mRNA': 'transcription of TetR'},
 {'cI mRNA': 'transcription of CI'}]

In [ ]:
# time params
ts = np.linspace(0, 15, 100)

# 1. run get_species(model=copasi).initial_concentration.to_dict().
y0 = [0.1, 10]

with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF'
    )

In [ ]:
dir(sol)


In [ ]:
sol

In [ ]:
vars(sol)

In [ ]:
sol.sol

In [ ]:
sol.t

In [ ]:
type(sol)

In [ ]:
sol.status == 'optimal'
